In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
import nltk
from collections import Counter
from tqdm.notebook import tqdm
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset
from transformers import pipeline, BertTokenizer

In [2]:
# paradigms in the BLiMP dataset
paradigms = ['adjunct_island', 
             'anaphor_gender_agreement', 
             'anaphor_number_agreement', 
             'animate_subject_passive', 
             'animate_subject_trans', 
             'causative', 
             'complex_NP_island', 
             'coordinate_structure_constraint_complex_left_branch', 
             'coordinate_structure_constraint_object_extraction', 
             'determiner_noun_agreement_1', 
             'determiner_noun_agreement_2', 
             'determiner_noun_agreement_irregular_1', 
             'determiner_noun_agreement_irregular_2', 
             'determiner_noun_agreement_with_adj_2', 
             'determiner_noun_agreement_with_adj_irregular_1', 
             'determiner_noun_agreement_with_adj_irregular_2', 
             'determiner_noun_agreement_with_adjective_1', 
             'distractor_agreement_relational_noun', 
             'distractor_agreement_relative_clause', 
             'drop_argument', 
             'ellipsis_n_bar_1', 
             'ellipsis_n_bar_2', 
             'existential_there_object_raising', 
             'existential_there_quantifiers_1', 
             'existential_there_quantifiers_2', 
             'existential_there_subject_raising', 
             'expletive_it_object_raising', 
             'inchoative', 
             'intransitive', 
             'irregular_past_participle_adjectives', 
             'irregular_past_participle_verbs', 
             'irregular_plural_subject_verb_agreement_1', 
             'irregular_plural_subject_verb_agreement_2', 
             'left_branch_island_echo_question', 
             'left_branch_island_simple_question', 
             #'matrix_question_npi_licensor_present', 
             'npi_present_1', 
             'npi_present_2', 
             'only_npi_licensor_present',
             'only_npi_scope', 
             'passive_1', 
             'passive_2', 
             'principle_A_c_command', 
             'principle_A_case_1', 
             'principle_A_case_2', 
             'principle_A_domain_1', 
             'principle_A_domain_2', 
             'principle_A_domain_3', 
             'principle_A_reconstruction', 
             'regular_plural_subject_verb_agreement_1', 
             'regular_plural_subject_verb_agreement_2', 
             'sentential_negation_npi_licensor_present', 
             'sentential_negation_npi_scope', 
             'sentential_subject_island', 
             'superlative_quantifiers_1', 
             'superlative_quantifiers_2', 
             'tough_vs_raising_1', 
             'tough_vs_raising_2', 
             'transitive', 
             'wh_island', 
             'wh_questions_object_gap', 
             'wh_questions_subject_gap', 
             'wh_questions_subject_gap_long_distance', 
             'wh_vs_that_no_gap', 
             'wh_vs_that_no_gap_long_distance', 
             'wh_vs_that_with_gap', 
             'wh_vs_that_with_gap_long_distance']

# A dictionary with abbveriations of lingustic terms for better display
phenomena = {
        "anaphor_agreement": "ANA AGR",
        "argument_structure": "ARG STR",
        "binding": "BINDING",
        "control_raising": "CTRL RAIS",
        "determiner_noun_agreement": "D-N AGR",
        "ellipsis": "ELLIPSIS",
        "filler_gap_dependency": "FILLER. GAP",
        "irregular_forms": "IRREGULAR",
        "island_effects": "ISLAND",
        "npi_licensing": "NPI",
        "quantifiers": "QUANTIFIERS",
        "subject_verb_agreement": "S-V AGR"
}

In [3]:
# Create a dataframe to hold data for each paradigm
blimp_df = pd.DataFrame()

# Iterate over each paradigm and filter the dataset
for paradigm in paradigms:
    # Load the dataset for the current paradigm
    subset = load_dataset('nyu-mll/BLiMP', name=paradigm)
    
    # Convert the dataset to a dataframe and add the paradigm column
    subset_df = pd.DataFrame(subset['train'])
    subset_df.rename(columns={'UID': 'paradigm'}, inplace=True)
    # Add the phenomenon column using the phenomena dictionary
    subset_df['phenomenon'] = subset_df['linguistics_term'].map(phenomena).fillna('EMPTY')

    # Print loading status
    print(f"\rLoading {paradigm}: {((paradigms.index(paradigm) + 1) / len(paradigms)) * 100:.2f}%{' ' * 30}", end='')
    
    # Append the subset dataframe to the main dataframe
    blimp_df = pd.concat([blimp_df, subset_df], ignore_index=True)

# Display the combined dataframe
blimp_df.head()

Loading wh_vs_that_with_gap_long_distance: 100.00%                                               

,sentence_good,sentence_bad,field,linguistics_term,paradigm,simple_LM_method,one_prefix_method,two_prefix_method,lexically_identical,pair_id,phenomenon
0,Who should Derek hug after shocking Richard?,Who should Derek hug Richard after shocking?,syntax,island_effects,adjunct_island,True,False,False,True,0,ISLAND
1,What had Theresa walked through while talking ...,What had Theresa walked through that high scho...,syntax,island_effects,adjunct_island,True,False,False,True,1,ISLAND
2,Who will Katherine discover without hiring Erin?,Who will Katherine discover Erin without hiring?,syntax,island_effects,adjunct_island,True,False,False,True,2,ISLAND
3,Who has Colleen aggravated before kissing Judy?,Who has Colleen aggravated Judy before kissing?,syntax,island_effects,adjunct_island,True,False,False,True,3,ISLAND
4,What could a lot of cats break while finding a...,What could a lot of cats break all convertible...,syntax,island_effects,adjunct_island,True,False,False,True,4,ISLAND


In [4]:
# Filter entries where sentence_good is equal to sentence_bad
identical_sentences = blimp_df[blimp_df['sentence_good'] == blimp_df['sentence_bad']]

# Print a report
print(f"Number of identical sentences: {len(identical_sentences)}")
if not identical_sentences.empty:
    print("Details of identical sentences:")
    print(identical_sentences[['sentence_good', 'paradigm']])
else:
    print("No identical sentences found.")

# Drop identical sentences from the dataframe
blimp_df = blimp_df[blimp_df['sentence_good'] != blimp_df['sentence_bad']]

Number of identical sentences: 7
Details of identical sentences:
                                           sentence_good            paradigm
39324               Douglas's senator was left by Susan.           passive_1
39810  Some grandmother of the cashier was left by Ja...           passive_1
43105  Mark hadn't forgotten about himself skated aro...  principle_A_case_2
43287  Leslie imagined herself skated around the hosp...  principle_A_case_2
43372  Susan thinks about herself skated around a river.  principle_A_case_2
43816  Amanda had imagined herself skated around a lo...  principle_A_case_2
43967  The cashiers think about themselves skated aro...  principle_A_case_2


In [24]:
# Initialize the BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

# Function to create a masked sentence with shared tokens and a single [MASK] for differences
def process_tokens(row):
    # Tokenize the good and bad sentences
    good_tokens = row['sentence_good'].split()
    bad_tokens = row['sentence_bad'].split()

    # Remove common tokens from the beginning
    common_start = []
    while good_tokens and bad_tokens and good_tokens[0] == bad_tokens[0]:
        common_start.append(good_tokens.pop(0))
        bad_tokens.pop(0)

    # Remove common tokens from the end
    common_end = []
    while good_tokens and bad_tokens and good_tokens[-1] == bad_tokens[-1]:
        common_end.insert(0, good_tokens.pop(-1))
        bad_tokens.pop(-1)


    # If the good or bad tokens are empty, add a common token to both
    if good_tokens == [] or bad_tokens == []:
        additional_tokens = common_end.pop(0)
        print("additional token:", additional_tokens)
        good_tokens.append(additional_tokens)
        bad_tokens.append(additional_tokens)

    # Collect the remaining tokens as the masked sentence
    common_tokens = common_start + ["[MASK]"] + common_end
    
    good_fillers = bert_tokenizer.tokenize(" ".join(good_tokens))
    bad_fillers = bert_tokenizer.tokenize(" ".join(bad_tokens))

    sentence_masked = " ".join(common_tokens)

    # If the last token of the good and bad fillers is the same, add it to the masked sentence (same punctuation with MASK at the end)
    if len(good_fillers) > 1 and len(bad_fillers) > 1: 
        if good_fillers[-1] == bad_fillers[-1]:
            remove_punct = good_fillers.pop(-1)
            sentence_masked += remove_punct
            bad_fillers.pop(-1)

    return sentence_masked, good_fillers, bad_fillers

In [ ]:
# Apply the function to the DataFrame
blimp_df[['sentence_masked', 'good_fillers', 'bad_fillers']] = blimp_df.apply(lambda row: pd.Series(process_tokens(row)), axis=1)

# Display a sample of the updated DataFrame
blimp_df[['sentence_masked', 'good_fillers', 'bad_fillers']].sample(10)

In [6]:
# Filter rows where good_fillers or bad_fillers are empty
empty_fillers = blimp_df[(blimp_df['good_fillers'].apply(len) == 0) | (blimp_df['bad_fillers'].apply(len) == 0)]

# Print the filtered rows
display(empty_fillers)

,sentence_good,sentence_bad,field,linguistics_term,paradigm,simple_LM_method,one_prefix_method,two_prefix_method,lexically_identical,pair_id,phenomenon,sentence_masked,good_fillers,bad_fillers
39403,All guys were talked about by a lot of libraries.,All guys were talked by a lot of libraries.,syntax,argument_structure,passive_1,True,False,False,False,403,ARG STR,All guys were talked [MASK] by a lot of librar...,[about],[]
57607,Samuel would escape from a waitress.,Samuel would escape a waitress.,syntax,argument_structure,transitive,True,False,True,False,607,ARG STR,Samuel would escape [MASK] a waitress.a,[from],[]
57944,The story hadn't disagreed with these actresses.,The story hadn't disagreed these actresses.,syntax,argument_structure,transitive,True,False,True,False,944,ARG STR,The story hadn't disagreed [MASK] these actres...,[with],[]


In [7]:
# Multiply the probabilities of the fillers to get the probability of the full sentence
def get_filler_probability(model, sentence, filler):
    total_probability = 1

    # Iterate over the fillers and calculate the probability of each word
    for i in range(0, len(filler)):
        word = filler[i]
        next_word = filler[i+1] if i+1 < len(filler) else ""

        # Calculate the probability of the current word
        word_probability = model(sentence, targets=[word])[0]['score']
        total_probability *= word_probability

        # Update the sentence with the filled word, pay attention to the ## tokens
        if word.startswith("##"):
            if next_word.startswith('##'):
                sentence = sentence.replace("[MASK]", word[2:] + "[MASK]")
            else:
                sentence = sentence.replace("[MASK]", word[2:] + " [MASK]")
        else:
            if next_word.startswith('##'):
                sentence = sentence.replace("[MASK]", word + "[MASK]")
            else:
                sentence = sentence.replace("[MASK]", word + " [MASK]")

        # Remove the [MASK] token if it is the last word
        if i+1 == len(filler):
            sentence = sentence.replace(" [MASK]", "")

    return total_probability ** (1 / len(filler)) if len(filler) > 0 else 0

In [8]:
bert_cased = pipeline("fill-mask", model="bert-base-cased")
#mbert_cased = pipeline("fill-mask", model="bert-base-multilingual-cased")

# different tokenizer needs different handling
#bibert_cased = pipeline("fill-mask", model="jhu-clsp/bibert-ende", tokenizer=BertTokenizer.from_pretrained("jhu-clsp/bibert-ende"))


# deploy_df = blimp_df.sample(1000)
deploy_df = blimp_df.copy()

# Calculate the probabilities for the good and bad fillers
deploy_df['bert_good_prob'] = deploy_df.apply(
    lambda row: get_filler_probability(bert_cased, row['sentence_masked'], row['good_fillers']), axis=1
)
deploy_df['bert_bad_prob'] = deploy_df.apply(
    lambda row: get_filler_probability(bert_cased, row['sentence_masked'], row['bad_fillers']), axis=1
)

# Calculate the prediction based on the probabilities
deploy_df['bert_prediction'] = deploy_df.apply(
    lambda row: 1 if row['bert_good_prob'] > row['bert_bad_prob'] else 0, axis=1
)

display(deploy_df[['sentence_masked', 'good_fillers', 'bad_fillers', 'bert_good_prob', 'bert_bad_prob', 'bert_prediction']])
deploy_df['bert_prediction'].value_counts()

# Save the dataframe to a CSV file
deploy_df.to_csv("bert_base_cased_prediction.csv", index=False)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls

,sentence_masked,good_fillers,bad_fillers,bert_good_prob,bert_bad_prob,bert_prediction
0,Who should Derek hug [MASK]?,"[after, shocking, Richard]","[Richard, after, shocking]",0.000078,0.000024,1
1,What had Theresa walked through [MASK]?,"[while, talking, about, that, high, school]","[that, high, school, while, talking, about]",0.009957,0.005844,1
2,Who will Katherine discover [MASK]?,"[without, hiring, Erin]","[Erin, without, hiring]",0.000013,0.000009,1
3,Who has Colleen aggravated [MASK]?,"[before, kissing, Judy]","[Judy, before, kissing]",0.000265,0.000082,1
4,What could a lot of cats break [MASK]?,"[while, finding, all, convertible, ##s]","[all, convertible, ##s, while, finding]",0.000062,0.000051,1
...,...,...,...,...,...,...
65995,A lot of boys remember [MASK] a report that di...,[what],[that],0.000803,0.005557,0
65996,Some committees knew [MASK] the actresses that...,[who],[that],0.000485,0.005854,0
65997,All governments forgot [MASK] these deer that ...,[what],[that],0.138488,0.314943,0
65998,Every patient didn't remember [MASK] these wai...,[who],[that],0.025648,0.014695,1
